**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/pose.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200909'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)




In [ ]:
# Let's do all of our imports here, too.
import numpy as np
import matplotlib.pyplot as plt, mpld3

from pydrake.all import (
    FindResourceOrThrow, set_log_level, ConnectMeshcatVisualizer, DepthCameraProperties, MakeRenderEngineVtk,RenderEngineVtkParams, RigidTransform, RollPitchYaw, Parser, AddMultibodyPlantSceneGraph, BaseField, DepthImageToPointCloud, DiagramBuilder, RgbdSensor)

if running_as_notebook:
    mpld3.enable_notebook()

# Simulating an RGB-D camera



In [ ]:
def DepthCameraDemoSystem():
    builder = DiagramBuilder()

    # If you have trouble finding resources, you can enable trace logging
    # to see how `FindResource*` is searching.
    set_log_level("trace")

    # Create the physics engine + scene graph.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    # Add a single object into it.
    Parser(plant, scene_graph).AddModelFromFile(
        FindResourceOrThrow(
            "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf"))
    # Add a rendering engine
    renderer = "my_renderer"
    scene_graph.AddRenderer(renderer,
                            MakeRenderEngineVtk(RenderEngineVtkParams()))
    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)
    meshcat.load()

    # Add a camera to the environment.
    pose = RigidTransform(RollPitchYaw(-0.2, 0.2, 0), [-.1, -0.1, -.5])
    properties = DepthCameraProperties(width=640,
                                       height=480,
                                       fov_y=np.pi / 4.0,
                                       renderer_name=renderer,
                                       z_near=0.1,
                                       z_far=10.0)
    camera = builder.AddSystem(
        RgbdSensor(parent_id=scene_graph.world_frame_id(),
                   X_PB=pose,
                   properties=properties,
                   show_window=False))
    camera.set_name("rgbd_sensor")
    builder.Connect(scene_graph.get_query_output_port(),
                    camera.query_object_input_port())

    # Export the camera outputs
    builder.ExportOutput(camera.color_image_output_port(), "color_image")
    builder.ExportOutput(camera.depth_image_32F_output_port(), "depth_image")

    # Add a system to convert the camera output into a point cloud
    to_point_cloud = builder.AddSystem(
        DepthImageToPointCloud(camera_info=camera.depth_camera_info(),
                               fields=BaseField.kXYZs | BaseField.kRGBs))
    builder.Connect(camera.depth_image_32F_output_port(),
                    to_point_cloud.depth_image_input_port())
    builder.Connect(camera.color_image_output_port(),
                    to_point_cloud.color_image_input_port())

    # Export the point cloud output.
    builder.ExportOutput(to_point_cloud.point_cloud_output_port(),
                         "point_cloud")

    diagram = builder.Build()
    diagram.set_name("depth_camera_demo_system")
    return diagram


In [ ]:
system = DepthCameraDemoSystem()

# Evaluate the camera output ports to get the images.
context = system.CreateDefaultContext()
system.Publish(context)
color_image = system.GetOutputPort("color_image").Eval(context)
depth_image = system.GetOutputPort("depth_image").Eval(context)

# Plot the two images.
plt.subplot(121)
plt.imshow(color_image.data)
plt.title('Color image')
plt.subplot(122)
plt.imshow(np.squeeze(depth_image.data))
plt.title('Depth image')
mpld3.display()
